Quick scripts to manipulate data after it's been downloaded

In [7]:
from PIL import Image
from glob import glob
from tqdm import tqdm
import os
from shutil import copyfile, move

Create version of data without ratings of 0,
since 0 ratings seem to be political jokes, not actual room ratings

In [14]:
os.mkdir('../data/rated_images_no0/')

In [101]:
for imname in dataset.imnames:
    rating = imname.split("_")[-1]
    rating = int(rating.split(".")[0])
    if rating > 0:
        name = imname.split('/')[-1]
        copyfile(imname, '../data/rated_images_no0/' + name)

Create low-res version (32x32 instead of 128x128).
Trained performance seems significantly lower, but may still be useful for experiments.

In [5]:
odir = 'rated_ims_lowres_no0/'
for imname in tqdm(glob('rated_images_no0/*.png')):
    im = Image.open(imname)
    im = im.resize((32, 32))
    name = imname.split('/')[-1]
    im.save(odir + name)

100%|██████████| 3011/3011 [00:40<00:00, 74.97it/s]


Split into train/test folders

I'll train off older tweets (higher numbers) and test on more recent

In [15]:
def test_train_split(folder, split = 1/4):
    """
    Move all images in folder into sub-folders train and test
    Split is the fraction of images moved to test
    Test uses the most recent tweets, train tweets are older
    """
    for name in ['test', 'train']:
        os.mkdir(os.path.join(folder, name))
    fnames = sorted(glob(os.path.join(folder, '*.png')))
    l_test = int(len(fnames) * split)
    for fname in fnames[:l_test]:
        name = fname.split('/')[-1]
        move(fname, os.path.join(folder, 'test', name))
    for fname in fnames[l_test:]:
        name = fname.split('/')[-1]
        move(fname, os.path.join(folder, 'train', name))

In [14]:
test_train_split('rated_images/')